In [ ]:
import numpy as np
import pandas as pd
rng = np.random.default_rng(12345)

from lymph.models import Unilateral
from lymixture import LymphMixture
from lymixture.utils import binom_pmf, late_binomial, normalize
from fixtures import (
    get_graph,
    get_patient_data,
    MODALITIES,
)
PARAMS_C1 = {
    "TtoII_spread": 0.5,
    "TtoIII_spread": 0.25,
    "TtoIV_spread": 0.1,
    "IItoIII_spread": 0.4,
    "IIItoIV_spread": 0.3,
    "late_p": 0.5,
}
PARAMS_C2 = {
    "TtoII_spread": 0.65,
    "TtoIII_spread": 0.15,
    "TtoIV_spread": 0.05,
    "IItoIII_spread": 0.5,
    "IIItoIV_spread": 0.4,
    "late_p": 0.5,
}

In [ ]:
graph = {
    ("tumor", "T"): ["II", "III"],
    ("lnl", "II"): ["III"],
    ("lnl", "III"): [],
}
patient_data = get_patient_data()
num_components = 2

mixture = LymphMixture(
    model_cls=Unilateral,
    model_kwargs={"graph_dict": graph},
    num_components=num_components,
)
data = pd.read_csv("data/mixture.csv", header=[0,1,2])
mixture.load_patient_data(data, split_by=("tumor", "1", "subsite"), mapping=lambda x: x)

mixture.set_modality("path", 1., 1.)

In [ ]:
mixture.set_distribution("early", binom_pmf(np.arange(11), 10, 0.3))
mixture.set_distribution("late", late_binomial)
mixture.get_all_distributions()

In [ ]:
from lymixture.em import expectation, maximization

params = {k: rng.uniform() for k in mixture.get_params()}
mixture.set_params(**params)
mixture.normalize_mixture_coefs()

latent = normalize(np.ones_like(mixture.get_resps()).T, axis=0).T

In [ ]:
mixture.get_mixture_coefs()

In [ ]:
params = maximization(mixture, latent)

In [ ]:
latent = expectation(mixture, params)

In [ ]:
def to_numpy(params: dict[str, float]) -> np.ndarray:
    return np.array([p for p in params.values()])

In [ ]:
converged = False
count = 0

while not converged:
    old_params = params
    params = maximization(mixture, latent)
    latent = expectation(mixture, params)
    converged = np.allclose(to_numpy(params), to_numpy(old_params))
    count += 1

mixture.get_params()

In [ ]:
count